## Config Variables and Load CSV Data

In [6]:
import csv

image_data_path = 'Data/sample'

# Images to use and offset (index, offset)
img_index = [(0, 0.0), (1, 0.2), (2, -0.2)]
measurement_index = 3

csv_data = []
with open(image_data_path + '/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        for i, off in img_index:
            csv_data.append({'img':row[i], 'center':float(row[measurement_index])+off, 'isMirror':True})
            csv_data.append({'img':row[i], 'center':float(row[measurement_index])+off, 'isMirror':False})

## Split data into train and validation

In [7]:
from sklearn.model_selection import train_test_split

train_samples, validation_samples = train_test_split(csv_data, test_size=0.2)

## Generator to read an pre-process image

In [8]:
import numpy as np
import cv2
from random import shuffle

''' Generator to load images '''

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: 
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = image_data_path+'/IMG/'+batch_sample['img'].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = batch_sample['center']
                
                if(batch_sample['isMirror']):
                    images.append(np.fliplr(center_image))
                    angles.append(-center_angle)
                else:
                    images.append(center_image)
                    angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

## NVIDIA Network Model

In [ ]:
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D
from keras.models import Sequential
from keras.models import Model
import sklearn

batch_size = 16

train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

image_shape = (160, 320, 3)

model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=image_shape, output_shape=image_shape))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= 
            len(train_samples), validation_data=validation_generator, 
            nb_val_samples=len(validation_samples), nb_epoch=3)

model.save('nvidia_gen.h5py')
print('DONE!')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch

Epoch 1/3
  478/38572 [..............................] - ETA: 415s - loss: 0.0209